# Downstream Analysis

This notebook should be used for downstream analysis of your OPS screen.
Cells marked with <font color='red'>SET PARAMETERS</font> contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

## <font color='red'>SET PARAMETERS</font>

### Fixed parameters for cluster module

- `CONFIG_FILE_PATH`: Path to a Brieflow config file used during processing. Absolute or relative to where workflows are run from.

In [1]:
CONFIG_FILE_PATH = "config/config.yml"

In [2]:
from pathlib import Path
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import yaml
import pandas as pd
import matplotlib as plt

from lib.cluster.cluster_analysis import (
    differential_analysis, 
    waterfall_plot, 
    two_feature_plot, 
    cluster_heatmap
)
from lib.cluster.phate_leiden_clustering import plot_phate_leiden_clusters
from lib.shared.metrics import get_all_stats

In [3]:
# load config file and determine root path
with open(CONFIG_FILE_PATH, "r") as config_file:
    config = yaml.safe_load(config_file)
    ROOT_FP = Path(config["all"]["root_fp"])

## Pipeline Statistics Report

This function analyzes the entire data processing pipeline from raw microscopy images to clustered perturbation profiles. It will:

1. **Preprocessing**: Count input files and generated image tiles
2. **SBS**: Analyze cell segmentation and barcode mapping success rates
3. **Phenotype**: Count cells and morphological features extracted
4. **Merge**: Calculate how many cells were successfully matched between SBS and phenotype data
5. **Aggregation**: Measure perturbation coverage and batch effect correction (per cell class/channel)
6. **Clustering**: Evaluate pathway enrichment and optimal clustering parameters

**Expected runtime: 5-15 minutes** depending on dataset size, as this function loads and analyzes large parquet files to calculate batch statistics and enrichment metrics.

The report will display progress updates as each stage is analyzed.

In [4]:
statistics = get_all_stats(config)

PIPELINE STATISTICS REPORT

[1/6] Gathering preprocessing statistics...

 PREPROCESSING STATISTICS:
   • ND2 input files: 38,670
   • SBS tiles generated: 22,620
   • Phenotype tiles generated: 8,025

[2/6] Gathering SBS statistics...

 SBS STATISTICS:
   • Total cells segmented: 5,193,329
   • Cells with barcode: 59.4% (3,086,044 cells)
   • Cells with unique gene mapping: 47.1% (2,444,937 cells)

[3/6] Gathering phenotype statistics...

 PHENOTYPE STATISTICS:
   • Total cells segmented: 5,369,034
   • Number of morphological features: 3,571

[4/6] Gathering merge statistics...

 MERGE STATISTICS:
   • Total cells processed: 2,674,264
   • Successfully mapped cells: 2,444,976
   • Average mapping rate: 91.4%

[5/6] Gathering aggregation statistics...
   (This may take a moment as it processes multiple cell class/channel combinations...)
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-A3_CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_PHALLOIDIN_WG

/lab/ops_analysis/cheeseman/kilimanjaro-analysis/brieflow/workflow/lib/shared/metrics.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_control_data["batch"] = (
/lab/barcheese01/mdiberna/miniconda3/envs/brieflow_kilimanjaro/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:111: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


Reading first 1,000 rows from brieflow_output/aggregate/parquets/CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_PHALLOIDIN_WGA__aligned.parquet


/lab/ops_analysis/cheeseman/kilimanjaro-analysis/brieflow/workflow/lib/shared/metrics.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_control_data["batch"] = (


Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-B3_CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-A3_CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-B1_CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-A2_CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-B2_CeCl-Interphase_ChCo-DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA__filtered.parquet


...

Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-A3_CeCl-all_ChCo-DAPI_VIMENTIN_ITGB1__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-B3_CeCl-all_ChCo-DAPI_VIMENTIN_ITGB1__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-B1_CeCl-all_ChCo-DAPI_VIMENTIN_ITGB1__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-A2_CeCl-all_ChCo-DAPI_VIMENTIN_ITGB1__filtered.parquet
Reading first 1,000 rows from brieflow_output/aggregate/parquets/P-1_W-B2_CeCl-all_ChCo-DAPI_VIMENTIN_ITGB1__filtered.parquet


/lab/ops_analysis/cheeseman/kilimanjaro-analysis/brieflow/workflow/lib/shared/metrics.py:355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_control_data["batch"] = (
/lab/barcheese01/mdiberna/miniconda3/envs/brieflow_kilimanjaro/lib/python3.11/site-packages/sklearn/feature_selection/_univariate_selection.py:111: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


Reading first 1,000 rows from brieflow_output/aggregate/parquets/CeCl-all_ChCo-DAPI_VIMENTIN_ITGB1__aligned.parquet


/lab/ops_analysis/cheeseman/kilimanjaro-analysis/brieflow/workflow/lib/shared/metrics.py:391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_control_data["batch"] = (



 AGGREGATION STATISTICS:

   Interphase cells - DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_PHALLOIDIN_WGA:
      • Distinct perturbations: 5,298
      • Median cells per perturbation: 258
      • Total cells (pre-filter): 2,945,339
      • Total cells (post-filter): 5,000
      • Total cells (aggregated): 1,412,773
      • PC features: 1371
      • Batch effect reduction: 0.3709 → 0.5242

   Interphase cells - DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA:
      • Distinct perturbations: 5,298
      • Median cells per perturbation: 258
      • Total cells (pre-filter): 2,945,339
      • Total cells (post-filter): 5,000
      • Total cells (aggregated): 1,412,773
      • PC features: 1232
      • Batch effect reduction: 0.4230 → 0.4816

   Interphase cells - DAPI_ACA_CENPT:
      • Distinct perturbations: 5,298
      • Median cells per perturbation: 258
      • Total cells (pre-filter): 2,945,339
      • Total cells (post-filter): 5,000
      • Total cells (aggregated): 1,412,773
      

In [5]:
# Load cell classes and channel combos
cluster_combo_fp = config["cluster"]["cluster_combo_fp"]
cluster_combos = pd.read_csv(cluster_combo_fp, sep="\t")

CHANNEL_COMBOS = list(cluster_combos["channel_combo"].unique())
print(f"Channel Combos: {CHANNEL_COMBOS}")

CELL_CLASSES = list(cluster_combos["cell_class"].unique())
print(f"Cell classes: {CELL_CLASSES}")

LEIDEN_RESOLUTION = list(cluster_combos["leiden_resolution"].unique())
print(f"Leiden resolution: {LEIDEN_RESOLUTION}")

Channel Combos: ['DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_PHALLOIDIN_WGA', 'DAPI_ACA_MITOTRACKER_VIMENTIN_CENPT_ITGB1_WGA', 'DAPI_ACA_CENPT', 'DAPI_CENPT', 'ACA_CENPT', 'DAPI', 'DAPI_MITOTRACKER_WGA', 'DAPI_VIMENTIN_PHALLOIDIN_ITGB1', 'DAPI_VIMENTIN_ITGB1']
Cell classes: ['Interphase', 'Mitotic', 'all']
Leiden resolution: [np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15)]


## <font color='red'>SET PARAMETERS</font>

### Cluster Selection for Analysis

Set these parameters to select the specific cluster data to analyze:
- `CHANNEL_COMBO`: Select from available channel combinations,
- `CELL_CLASS`: Select from available cell classes,
- `LEIDEN_RESOLUTION`: Select from available Leiden resolutions,

These parameters determine which folder of cluster data will be analyzed.

In [6]:
CHANNEL_COMBO = "ACA_CENPT"
CELL_CLASS = "all"
LEIDEN_RESOLUTION = 14

In [7]:
aggregate_file = ROOT_FP / "aggregate" / "tsvs" / f"CeCl-{CELL_CLASS}_ChCo-{CHANNEL_COMBO}__feature_table.tsv"
print(f"Aggregate file: {aggregate_file}")

if not aggregate_file.exists():
    print(f"Aggregate file does not exist: {aggregate_file}")
else:
    print(f"Aggregate file found")

cluster_path = ROOT_FP / "cluster" / CHANNEL_COMBO / CELL_CLASS / str(LEIDEN_RESOLUTION)
print(f"Cluster path: {cluster_path}")

if not cluster_path.exists():
    print(f"Cluster directory does not exist: {cluster_path}")
else:
    print(f"Cluster directory found")

Aggregate file: brieflow_output/aggregate/tsvs/CeCl-all_ChCo-ACA_CENPT__feature_table.tsv
Aggregate file found
Cluster path: brieflow_output/cluster/ACA_CENPT/all/14
Cluster directory found


## Feature Plot Analysis

This section generates visualizations to explore the phenotypic effects of gene perturbations in your screen. The plots will help you:

1. **Differential Feature Analysis**: Identify genes with significant phenotypic changes vs. controls
2. **Waterfall Plots**: Rank genes by their effect on specific features of interest
3. **Two-Feature Plots**: Discover relationships between different phenotypic measurements
4. **Heatmaps**: Visualize patterns across multiple features and gene sets simultaneously

The interactive analysis allow you to customize each visualization for your specific biological questions.

In [ ]:
cluster_file = cluster_path / "phate_leiden_clustering.tsv"
cluster_df = pd.read_csv(cluster_file, sep="\t")
display(cluster_df)

In [ ]:
aggregate_df = pd.read_csv(aggregate_file, sep="\t")
display(aggregate_df)

## <font color='red'>SET PARAMETERS</font>

### Cluster Selection for Visualization

Set these parameters to select the specific cluster to analyze:
- `CLUSTER_ID`: The cluster of interest to generate plots.

In [ ]:
CLUSTER_ID = None

In [ ]:
print(f"\n{'='*50}")
print(f"Analyzing Cluster {CLUSTER_ID}")
cluster_genes = cluster_df[cluster_df['cluster'] == CLUSTER_ID][config["aggregate"]["perturbation_name_col"]]
print(f"Genes in Cluster {CLUSTER_ID}:" ,", ".join(cluster_genes.unique()))
print(f"{'='*50}")

# Run differential analysis using robust z-score
diff_results = differential_analysis(
    feature_df=aggregate_df,  
    cluster_df=cluster_df,    
    cluster_id=CLUSTER_ID,
    control_type="nontargeting",  
    control_label=config["aggregate"]["control_key"],
    use_nonparametric=True,  
    normalize_method="robust_zscore"
)

# Display results in a more user-friendly format
print("\nTop Upregulated Features:")
up_df = diff_results['top_up'][['feature', 'robust_zscore', 'p_value', 'median_test', 'median_control']]
up_df = up_df.rename(columns={
    'robust_zscore': 'Z-score',
    'p_value': 'p-value',
    'median_test': 'Median (test)',
    'median_control': 'Median (control)'
})
display(up_df.style.format({
    'Z-score': '{:.2f}',
    'p-value': '{:.2e}',
    'Median (test)': '{:.3f}',
    'Median (control)': '{:.3f}'
}))

print("\nTop Downregulated Features:")
down_df = diff_results['top_down'][['feature', 'robust_zscore', 'p_value', 'median_test', 'median_control']]
down_df = down_df.rename(columns={
    'robust_zscore': 'Z-score',
    'p_value': 'p-value',
    'median_test': 'Median (test)',
    'median_control': 'Median (control)'
})
display(down_df.style.format({
    'Z-score': '{:.2f}',
    'p-value': '{:.2e}',
    'Median (test)': '{:.3f}',
    'Median (control)': '{:.3f}'
}))

## <font color='red'>SET PARAMETERS</font>

### Feature Selection for Visualization

Set these parameters to select the specific cluster to analyze:
- `FEATURES_TO_ANALYZE`: The features found in the differential analysis to generate plots for.
- `GENES_TO_LABEL`: The genes within the cluster to label on the plots.

In [ ]:
FEATURES_TO_ANALYZE = None
GENES_TO_LABEL = None

In [ ]:
# Create waterfall plots for selected features
print("\n--- Waterfall Plots for Selected Features ---")
for feature in FEATURES_TO_ANALYZE:
    print(f"\nPlotting: {feature}")
    waterfall_plot(
        feature_df=aggregate_df, 
        feature=feature,
        cluster_df=cluster_df,
        cluster_id=CLUSTER_ID,
        nontargeting_pattern=config["aggregate"]["control_key"],
        title=f"Cluster {CLUSTER_ID}: {feature}",
        label_genes=GENES_TO_LABEL
    )

# Create two-feature plots for combinations
print("\n--- Two-Feature Plots ---")
# Generate all unique pairs of features
feature_pairs = [(FEATURES_TO_ANALYZE[i], FEATURES_TO_ANALYZE[j]) 
                 for i in range(len(FEATURES_TO_ANALYZE)) 
                 for j in range(i+1, len(FEATURES_TO_ANALYZE))]

for feature1, feature2 in feature_pairs:
    print(f"\nPlotting: {feature1} vs {feature2}")
    two_feature_plot(
        feature_df=aggregate_df,
        x=feature1,
        y=feature2,
        cluster_df=cluster_df,
        cluster_id=CLUSTER_ID,
        nontargeting_pattern=config["aggregate"]["control_key"],
        title=f"Cluster {CLUSTER_ID}: {feature1} vs {feature2}",
        label_genes=GENES_TO_LABEL
    )

# Create heatmap with all differential features
print("\n--- Heatmap of Differential Features ---")
# Get top features from differential analysis
top_up = diff_results['top_up']['feature'].tolist() if not diff_results['top_up'].empty else []
top_down = diff_results['top_down']['feature'].tolist() if not diff_results['top_down'].empty else []
diff_features = top_up + top_down

fig, ax, heatmap_data = cluster_heatmap(
    feature_df=aggregate_df,
    cluster_df=cluster_df,
    cluster_ids=[CLUSTER_ID],
    features=diff_features,
    perturbation_name_col=config["aggregate"]["perturbation_name_col"],
    z_score="global",
    title=f"Cluster {CLUSTER_ID}: Top Differential Features",
)

# PHATE plot for the cluster
print("\n--- PHATE Plot for Cluster ---")
phate_fig = plot_phate_leiden_clusters(
    phate_leiden_clustering=cluster_df,
    perturbation_name_col=config["aggregate"]["perturbation_name_col"],
    control_key=config["aggregate"]["control_key"],
    clusters_of_interest=[CLUSTER_ID],
)

## Mozzarellm: LLM-based Gene Cluster Analysis

### Overview
[Mozzarellm](https://github.com/cheeseman-lab/mozzarellm) is a Python package that leverages Large Language Models (LLMs) to analyze gene clusters for pathway identification and novel gene discovery. This notebook guides you through the process of:

1. **Loading and reshaping gene cluster data** from your OPS screen
2. **Analyzing gene clusters with LLMs** to identify biological pathways
3. **Categorizing genes** as established pathway members, uncharacterized, or having novel potential roles
4. **Prioritizing candidates** for experimental validation

### Prerequisites

You need to install the mozzarellm package in your Brieflow environment:

```bash
pip install git+https://github.com/cheeseman-lab/mozzarellm.git
```

Mozzarellm requires API keys to access LLM services. You need at least one of these keys:

- **OpenAI API Key**: Required for GPT models (gpt-4o, gpt-4.5, etc.)
- **Anthropic API Key**: Required for Claude models (claude-3-7-sonnet, etc.)
- **Google API Key**: Required for Gemini models (gemini-2.0-pro, etc.)

These keys provide access to paid API services, and usage will incur costs based on the number of tokens processed. The cost per analysis varies by model but typically ranges from $0.01-$0.10 per cluster, depending on cluster size and model choice. For this reason, we only run these analyses on a specific chosen Leiden resolution, rather than across all of the generated possible resolutions.

In [10]:
import os
from mozzarellm import analyze_gene_clusters, reshape_to_clusters

In [ ]:
# Set API keys
os.environ["OPENAI_API_KEY"] = "your_openai_key_here"
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_key_here"  
os.environ["GOOGLE_API_KEY"] = "your_google_key_here"

In [12]:
RESULTS_DIR = cluster_path / "mozzarellm_analysis"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Results will be saved to: {RESULTS_DIR}")

Results will be saved to: brieflow_output/cluster/ACA_CENPT/all/14/mozzarellm_analysis


In [13]:
cluster_file = cluster_path / "phate_leiden_clustering.tsv"
cluster_df = pd.read_csv(cluster_file, sep="\t")
display(cluster_df)

,gene_symbol_0,cell_count,PHATE_0,PHATE_1,cluster,uniprot_entry,uniprot_function,uniprot_link,mean_potential_to_nontargeting,normalized_potential_to_nontargeting
0,RPL34,162,-0.103713,-0.022562,0,P49207,Component of the large ribosomal subunit (PubM...,https://www.uniprot.org/uniprotkb/P49207/entry,6.318851,0.814492
1,RPL5,325,-0.121327,-0.029349,0,P46777,"Component of the ribosome, a large ribonucleop...",https://www.uniprot.org/uniprotkb/P46777/entry,7.415016,0.963383
2,RPL8,166,-0.102111,-0.023354,0,P62917,Component of the large ribosomal subunit. The ...,https://www.uniprot.org/uniprotkb/P62917/entry,6.229079,0.802299
3,DDX27,319,-0.103186,-0.023049,0,Q96GQ7,Probable ATP-dependent RNA helicase. Component...,https://www.uniprot.org/uniprotkb/Q96GQ7/entry,6.303440,0.812399
4,MAK16,611,-0.091161,-0.017100,0,Q9BXY0,NaN,https://www.uniprot.org/uniprotkb/Q9BXY0/entry,5.516524,0.705513
...,...,...,...,...,...,...,...,...,...,...
5293,EMC1,783,0.000069,-0.001782,485,Q8N766,Part of the endoplasmic reticulum membrane pro...,https://www.uniprot.org/uniprotkb/Q8N766/entry,0.477448,0.021061
5294,MTPAP,330,0.008110,-0.000448,485,Q9NVV4,Polymerase that creates the 3' poly(A) tail of...,https://www.uniprot.org/uniprotkb/Q9NVV4/entry,0.520986,0.026975
5295,OR1E2,424,0.005374,0.000915,485,P47887,Odorant receptor. {ECO:0000305}.,https://www.uniprot.org/uniprotkb/P47887/entry,0.401281,0.010715
5296,TTC37,572,0.008340,-0.002477,485,Q6PGP7,"Component of the SKI complex, a multiprotein c...",https://www.uniprot.org/uniprotkb/Q6PGP7/entry,0.577484,0.034649


## <font color='red'>SET PARAMETERS</font>: 

### Data Reshaping

Configure gene clustering parameters:
- `GENE_COL`: Column containing gene identifiers
- `CLUSTER_COL`: Column containing cluster assignments
- `UNIPROT_COL`: Column with UniProt annotations

These parameters control how gene-level data is converted to cluster-level data. These are set to default values that will be established in upstream notebooks/snakemakes, and usually don't need to be manipulated.

In [14]:
GENE_COL = config["aggregate"]["perturbation_name_col"]
CLUSTER_COL = "cluster"
UNIPROT_COL = "uniprot_function"

In [15]:
llm_cluster_df, llm_uniprot_df = reshape_to_clusters(
    input_df=cluster_df, 
    gene_col=GENE_COL,
    cluster_col=CLUSTER_COL,
    uniprot_col=UNIPROT_COL, 
    verbose=True
)
display(llm_cluster_df)
display(llm_uniprot_df)

Using provided DataFrame with 5298 rows
Found 5298 genes across 486 clusters
Extracting gene features from uniprot_function column


,cluster_id,genes
0,0,RPL34;RPL5;RPL8;DDX27;MAK16;RPL35;AAMP;POLR3E;...
1,1,RPS17;RPS21;ZNHIT6;NOP56;RPS27A;UTP18;NOP58;NO...
2,2,LSM6;SNRPD1;SNRPB;SNIP1;PUF60;ARID3C;TUT1;U2AF...
3,3,H3C14;PAXBP1;RPA1;PCNA;MMS22L;TICRR;CDC45;ATAD...
4,4,CRNKL1;SF3B4;AQR;DDX41;SF3A2;SPATA31A7;SLU7;IL...
...,...,...
481,481,WDR7;WDR83OS;FAF2;DIS3;SLC22A25;RPL36AL
482,482,NYAP1;USP5;TRIM47;DPPA3;PIGH;NDUFA2
483,483,YRDC;TMEM209;KCND3;ERCC4;CPEB2;TSSK2
484,484,nontargeting_ACCCTATCCAA;FOXN4;ZBTB45;MRPL40;H...


,gene_symbol_0,uniprot_function
0,RPL34,Component of the large ribosomal subunit (PubM...
1,RPL5,"Component of the ribosome, a large ribonucleop..."
2,RPL8,Component of the large ribosomal subunit. The ...
3,DDX27,Probable ATP-dependent RNA helicase. Component...
4,MAK16,
...,...,...
5293,EMC1,Part of the endoplasmic reticulum membrane pro...
5294,MTPAP,Polymerase that creates the 3' poly(A) tail of...
5295,OR1E2,Odorant receptor. {ECO:0000305}.
5296,TTC37,"Component of the SKI complex, a multiprotein c..."


## <font color='red'>SET PARAMETERS</font>: 
### LLM Analysis Configuration

- `MODEL_NAME`: LLM to use for analysis. Usable models include:
  - OpenAI: `o4-mini`, `o3-mini`, `gpt-4.1`, `gpt-4o`
  - Anthropic: `claude-3-7-sonnet-20250219`, `claude-3-5-haiku-20241022`
  - Google: `gemini-2.5-pro-preview-03-25`, `gemini-2.5-flash-preview-04-17`
- `CONFIG_DICT`: Configuration file for the LLM model
- `SCREEN_CONTEXT`: Context for the analysis and how to evaluate _clusters_
- `CLUSTER_ANALYSIS_PROMPT`: Context for the analysis and how to evaluate _genes within clusters_

Mozzarellm includes optimized [configurations](https://github.com/cheeseman-lab/mozzarellm/blob/main/mozzarellm/configs.py) and [prompts](https://github.com/cheeseman-lab/mozzarellm/blob/main/mozzarellm/prompts.py) you can import as shown below.

Custom text files can also be used by setting `screen_context_path` and `cluster_analysis_prompt_path` parameters.

In [16]:
from mozzarellm.prompts import ROBUST_SCREEN_CONTEXT, ROBUST_CLUSTER_PROMPT
from mozzarellm.configs import DEFAULT_ANTHROPIC_CONFIG

In [ ]:
# Set up model configs
MODEL_NAME = "claude-3-7-sonnet-20250219"
CONFIG_DICT = DEFAULT_ANTHROPIC_CONFIG
SCREEN_CONTEXT = ROBUST_SCREEN_CONTEXT
CLUSTER_ANALYSIS_PROMPT = ROBUST_CLUSTER_PROMPT

In [19]:
# Run LLM analysis
anthropic_results = analyze_gene_clusters(
    # Input data options
    input_df=llm_cluster_df,
    # Model and configuration
    model_name=MODEL_NAME,
    config_dict=CONFIG_DICT,
    # Analysis context and prompts
    screen_context=SCREEN_CONTEXT,
    cluster_analysis_prompt=CLUSTER_ANALYSIS_PROMPT,
    # Gene annotations
    gene_annotations_df=llm_uniprot_df,
    # Processing options
    batch_size=1,
    # Output options
    output_file=f"{RESULTS_DIR}/{MODEL_NAME}",
    save_outputs=True,
    outputs_to_generate=["json", "clusters", "flagged_genes"],
)

Loaded data with 486 rows and columns: ['cluster_id', 'genes']
Created annotations dictionary with 5298 entries from DataFrame


Processing clusters:   0%|          | 0/486 [00:00<?, ?it/s]

Using provided template string
Appending output format instructions to template
Added 79 gene feature descriptions to prompt


Processing clusters:   0%|          | 1/486 [00:30<4:10:01, 30.93s/it]

Using provided template string
Appending output format instructions to template
Added 71 gene feature descriptions to prompt


...

INFO:cluster_analysis_20250714_125812.log:Anthropic API call successful: model=claude-3-7-sonnet-20250219
INFO:cluster_analysis_20250714_125812.log:Success for cluster 483
Processing clusters: 100%|█████████▉| 484/486 [2:01:16<00:22, 11.38s/it]INFO:cluster_analysis_20250714_125812.log:Using Anthropic Claude API


Using provided template string
Appending output format instructions to template
Added 6 gene feature descriptions to prompt


INFO:cluster_analysis_20250714_125812.log:Anthropic API call successful: model=claude-3-7-sonnet-20250219
INFO:cluster_analysis_20250714_125812.log:Success for cluster 484
INFO:cluster_analysis_20250714_125812.log:Saved progress for 485 clusters
Processing clusters: 100%|█████████▉| 485/486 [2:01:27<00:11, 11.36s/it]INFO:cluster_analysis_20250714_125812.log:Using Anthropic Claude API


Using provided template string
Appending output format instructions to template
Added 5 gene feature descriptions to prompt


INFO:cluster_analysis_20250714_125812.log:Anthropic API call successful: model=claude-3-7-sonnet-20250219
INFO:cluster_analysis_20250714_125812.log:Success for cluster 485
Processing clusters: 100%|██████████| 486/486 [2:01:39<00:00, 15.02s/it]
INFO:cluster_analysis_20250714_125812.log:Completed analysis for 486 clusters with data saved to disk
